In [9]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.8 kB ? eta -:--:--
     ---------------------- ---------------- 92.2/162.8 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 162.8/162.8 kB 1.4 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
   - -------------------------------------- 0.2/7.7 MB 5.0 MB/s eta 0:00:02
   --- ------------------------------------ 0.6/7.7 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.7 MB 6.8 MB/s eta 0:00:01
   ------- -------------------------------- 1.4/7.7 MB 7.6 MB/s eta 0:00:01
   ---------- ----------------------------- 2.0/7.7 MB 8.5 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/7.7 MB 9.8 MB/s eta 0:00:01
   ----------------- -------

In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

data = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_month.xlsx")
ts = ['Jan_2022', "Feb_2022", "Mar_2022", "Apr_2022", "May_2022", "Jun_2022", "Jul_2022", "Aug_2022", "Sep_2022", "Oct_2022", "Nov_2022", "Dec_2022", 'Jan_2023', "Feb_2023", "Mar_2023", "Apr_2023", "May_2023", "Jun_2023", "Jul_2023", "Aug_2023", "Sep_2023", "Oct_2023", "Nov_2023", "Dec_2023"]
data.fillna(0, inplace=True)
df = data[ts]
# Preprocess the data
# For LSTM, we need to reshape the data into sequences
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values.reshape(-1, 1))

# Define the window size
window_size = 12  # 12 previous months

# Create sequences of data
X, y = create_sequences(scaled_data, window_size)

# Split the data into training and validation sets
# For simplicity, let's use the first 80% of the data for training
# and the remaining 20% for validation
split_index = int(0.8 * len(X))
X_train, X_valid = X[:split_index], X[split_index:]
y_train, y_valid = y[:split_index], y[split_index:]

# Build the LSTM model
model = Sequential([
    LSTM(units=50, activation='relu', input_shape=(window_size, X.shape[2])),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

# Plot training history (optional)
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Make predictions
predictions = model.predict(X_valid)

predictions = predictions.reshape(-1, 1)

# Inverse scaling of predictions
predictions = scaler.inverse_transform(predictions)

# Evaluate the model (optional)
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_valid, predictions)
mae = mean_absolute_error(y_valid, predictions)

print("Mean Squared Error (MSE):", mse)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11048\179317660.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna(0, inplace=True)


[0.00423674]
[0.00276259]
[0.004024]
[0.]
[0.]
[0.00838492]
[0.01481579]
[0.00269557]
[0.00120438]
[0.00306906]
[0.00894039]
[0.00646225]
[0.00174616]
[0.00743626]
[0.0048879]
[0.00809645]
[0.00197009]
[0.0036569]
[0.00434955]
[0.00347297]
[0.00632072]
[0.00207983]
[0.00310435]
[0.00409729]
[0.]
[0.00077913]
[0.00011118]
[0.00094528]
[0.00030083]
[0.]
[0.00158769]
[0.0014438]
[0.00054853]
[0.0009102]
[0.]
[0.00024955]
[0.00010693]
[0.00027824]
[5.26076561e-05]
[0.00027327]
[0.00017047]
[6.05673003e-06]
[0.]
[0.0004397]
[0.]
[0.00037168]
[0.00016851]
[0.00013047]
[0.00527013]
[0.00994956]
[0.01216426]
[0.00292518]
[0.00248838]
[0.]
[0.01571721]
[0.00019291]
[0.0041415]
[0.0077108]
[0.01188888]
[0.01503967]
[0.00367236]
[0.00249088]
[0.00220159]
[0.00146969]
[0.00314916]
[0.00335563]
[0.00439475]
[0.00382957]
[0.00147792]
[0.]
[0.00232818]
[0.00219622]
[0.00043958]
[0.00041411]
[0.00110172]
[0.00073503]
[0.00076064]
[0.00013779]
[0.00011217]
[0.00034102]
[0.00077076]
[0.]
[0.]
[0.]
[0.]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [15]:
# Get the most recent 12 months of data
recent_data = df.iloc[-window_size:]

# Scale the recent data
scaled_recent_data = scaler.transform(recent_data.values.reshape(-1, 1))

# Create sequences for prediction
X_pred, _ = create_sequences(scaled_recent_data, window_size)

# Make predictions
predictions_next_12_months = model.predict(X_pred)

# Reshape predictions
predictions_next_12_months = predictions_next_12_months.reshape(-1, 1)

# Inverse scaling of predictions
predictions_next_12_months = scaler.inverse_transform(predictions_next_12_months)

print("Predictions for the next 12 months:")
for i, pred in enumerate(predictions_next_12_months):
    print(f"Month {i+1}: {pred[0]}")



9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predictions for the next 12 months:
Month 1: 5063.671875
Month 2: 5049.9638671875
Month 3: 6366.96435546875
Month 4: 4575.1669921875
Month 5: 5272.49462890625
Month 6: 5718.62841796875
Month 7: 5753.12353515625
Month 8: 6266.1416015625
Month 9: 5612.61767578125
Month 10: 5487.93798828125
Month 11: 3989.17919921875
Month 12: 4854.3603515625
Month 13: 7866.576171875
Month 14: 5537.521484375
Month 15: 4837.26953125
Month 16: 5272.005859375
Month 17: 3987.854248046875
Month 18: 3791.302978515625
Month 19: 2975.2724609375
Month 20: 3200.53466796875
Month 21: 3339.47119140625
Month 22: 4414.76123046875
Month 23: 3725.280029296875
Month 24: 2928.74169921875
Month 25: 2854.897705078125
Month 26: 3655.054931640625
Month 27: 4252.63232421875
Month 28: 3804.34228515625
Month 29: 3729.044677734375
Month 30: 3308.431884765625
Month 31: 2690.93359375
Month 32: 3406.2958984375
Month 33: 2679.42529296875
Month 34: 2321.919921875
Month 35: 3646.205078125
Month 36: 